<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_65.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model


In [1]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-15/checkpoint-20000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-15/checkpoint-20000")

loading configuration file /content/Model/Models-Train-15/checkpoint-20000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "@accessori_saldatore",
    "1": "@accessori_trapano",
    "2": "@armadio_cassettiera",
    "3": "@bancale_cargo",
    "4": "@banco",
    "5": "@brugola",
    "6": "@cacciavite_giravite_cercafase",
    "7": "@carrello",
    "8": "@carta-vetrata_carta-abrasiva",
    "9": "@cassaforte",
    "10": "@catena",
    "11": "@cazzuola_frattone",
    "12": "@cerniera_bandella",
    "13": "@chiave-a-bussola_tubo",
    "14": "@chiave_inglese_poligonale",
    "15": "@chiavistello",
    "16": "@chiodo_rivetto",
    "17": "@colla_silicone",
    "18": "@collare",
    "19": "@coltello",
    "20": "@estrattore",
    "21": "@fo

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-ferramenta-val", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-ferramenta/images-val", split="train")

In [ ]:
ds

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filter_funtion)

In [ ]:
ds

In [ ]:
for i in range(0,150):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [4]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [5]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-ferramenta-val", split="train")

Resolving data files:   0%|          | 0/21869 [00:00<?, ?it/s]

In [6]:
ds_f

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [7]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.7

In [8]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

Counter({51: 2200, 21: 1926, 1: 1879, 14: 1555, 43: 1352, 33: 1196, 6: 1069, 13: 842, 27: 697, 28: 695, 8: 663, 9: 642, 23: 634, 41: 522, 26: 450, 48: 379, 2: 345, 31: 331, 39: 327, 22: 281, 24: 270, 44: 254, 36: 238, 17: 206, 40: 203, 25: 196, 5: 176, 49: 159, 42: 157, 7: 148, 35: 148, 29: 146, 37: 146, 38: 142, 10: 136, 4: 132, 47: 116, 12: 112, 50: 106, 30: 85, 46: 80, 34: 73, 0: 59, 18: 59, 32: 54, 16: 51, 3: 48, 19: 43, 45: 37, 15: 36, 11: 35, 20: 33})


In [9]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: []}


In [10]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [11]:
# selected_indices

In [12]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [13]:
# selected_indices

In [14]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [15]:
fused_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 15336
})

In [16]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-ferramenta/images-val", split="train")

Resolving data files:   0%|          | 0/43738 [00:00<?, ?it/s]

In [17]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [18]:
ds_imgs = ds_joint.filter(filter_funtion)

In [19]:
ds_imgs

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [20]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [21]:
all_indices = range(len(ds_imgs))
print(all_indices)

range(0, 21869)


In [22]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [23]:
# remaining_indices

In [24]:
missing_ds = ds_imgs.select(remaining_indices)

In [25]:
missing_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 6533
})

In [26]:
# Combine Both Datasets

In [27]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [28]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

# Pre-Processing and Validation Loop

In [29]:
labels = ds.features["label"]
labels

ClassLabel(names=['@accessori_saldatore', '@accessori_trapano', '@armadio_cassettiera', '@bancale_cargo', '@banco', '@brugola', '@cacciavite_giravite_cercafase', '@carrello', '@carta-vetrata_carta-abrasiva', '@cassaforte', '@catena', '@cazzuola_frattone', '@cerniera_bandella', '@chiave-a-bussola_tubo', '@chiave_inglese_poligonale', '@chiavistello', '@chiodo_rivetto', '@colla_silicone', '@collare', '@coltello', '@estrattore', '@forbice_cesoia', '@lama_seghetto_sega', '@lame_sega_circolare', '@lima_grattuggia_raspa', '@livella', '@lucchetto', '@maniglia_pomello', '@martello_mazzetta_ascia', '@metro_flessometro', '@morsetto_strettoio', '@nastro', '@pennello', '@pinza_punzonatrice_giratubo', '@pinzetta', '@pistola', '@rubinetto_miscelatore_valvola', '@ruota', '@scaffale', '@scala', '@scalpello', '@sega-a-tazza', '@seghetto_sega', '@segnaletica', '@serratura_cilindro', '@spatola_raschietto', '@spazzola', '@staffa_squadretta', '@tassello', '@torcia-faro', '@vernice', '@vite_bullone'], id=Non

In [30]:
labels.int2str(ds[532]["label"])

'@accessori_trapano'

In [31]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [32]:
dataset = ds.with_transform(transform)

In [33]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [34]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [35]:
labels = ds.features["label"].names
labels

['@accessori_saldatore',
 '@accessori_trapano',
 '@armadio_cassettiera',
 '@bancale_cargo',
 '@banco',
 '@brugola',
 '@cacciavite_giravite_cercafase',
 '@carrello',
 '@carta-vetrata_carta-abrasiva',
 '@cassaforte',
 '@catena',
 '@cazzuola_frattone',
 '@cerniera_bandella',
 '@chiave-a-bussola_tubo',
 '@chiave_inglese_poligonale',
 '@chiavistello',
 '@chiodo_rivetto',
 '@colla_silicone',
 '@collare',
 '@coltello',
 '@estrattore',
 '@forbice_cesoia',
 '@lama_seghetto_sega',
 '@lame_sega_circolare',
 '@lima_grattuggia_raspa',
 '@livella',
 '@lucchetto',
 '@maniglia_pomello',
 '@martello_mazzetta_ascia',
 '@metro_flessometro',
 '@morsetto_strettoio',
 '@nastro',
 '@pennello',
 '@pinza_punzonatrice_giratubo',
 '@pinzetta',
 '@pistola',
 '@rubinetto_miscelatore_valvola',
 '@ruota',
 '@scaffale',
 '@scala',
 '@scalpello',
 '@sega-a-tazza',
 '@seghetto_sega',
 '@segnaletica',
 '@serratura_cilindro',
 '@spatola_raschietto',
 '@spazzola',
 '@staffa_squadretta',
 '@tassello',
 '@torcia-faro',
 '@v

In [36]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [37]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [38]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [39]:
batch_size = 32

In [40]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [41]:
model = model.to(device)
model.eval()
predictions, labels = [], []
# valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    # loss = outputs.loss
    # valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
# AUROC_score = roc_auc_score(labels, predictions)

In [43]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.9499291234167085
precision:  0.9500391544357965
f1_score:  0.9497480139518626


In [44]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.79      0.51      0.62        59
           1       0.97      0.97      0.97      1879
           2       0.95      0.99      0.97       345
           3       0.94      0.98      0.96        48
           4       0.94      0.98      0.96       132
           5       0.89      0.88      0.88       176
           6       0.97      0.98      0.98      1069
           7       0.92      0.94      0.93       148
           8       0.96      0.97      0.97       663
           9       1.00      0.99      0.99       642
          10       0.91      0.93      0.92       136
          11       0.89      0.91      0.90        35
          12       0.92      0.92      0.92       112
          13       0.95      0.96      0.95       842
          14       0.97      0.98      0.98      1555
          15       0.88      0.78      0.82        36
          16       0.92      0.92      0.92        51
          17       0.89    

In [45]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[  30    6    0 ...    0    0    1]
 [   1 1832    0 ...    0    0    1]
 [   1    0  340 ...    0    0    0]
 ...
 [   0    2    0 ...  134    0    0]
 [   1    0    0 ...    0   99    0]
 [   0    2    0 ...    0    0 2168]]
